In [1]:
from sympy import Matrix, S, latex
from sympy.abc import x, y, z
from lmi_sdp.lm import lm_sym_expanded
from lmi_sdp.lmi import ShapeError, NonSymmetricMatrixError, LMI_PSD, \
    LMI_NSD, LMI_PD, LMI_ND, init_lmi_latex_printing

### test_LMI_PSD()

In [2]:
m = Matrix([[x, y], [y, z+1]])
lmi = LMI_PSD(m)
assert lmi.lhs == lmi.gts == m
assert lmi.rhs == lmi.lts == S(0)

c = Matrix([[0, 1], [1, 2]])
lmi = LMI_PSD(m, c)
assert lmi.lhs == lmi.gts == m
assert lmi.rhs == lmi.lts == c

### test_LMI_canonical()

In [3]:
m = Matrix([[x, y], [y, z+1]])
c = Matrix([[0, 1], [1, 2]])
can_lhs = m-c
can_rhs = 0

can = LMI_PSD(m, c).canonical()
assert can.lhs == can.gts == can_lhs
assert can.rhs == can.lts == can_rhs

can = LMI_PSD(0, c-m).canonical()
assert can.lhs == can.gts == can_lhs
assert can.rhs == can.lts == can_rhs

can = LMI_PD(m, c).canonical()
assert can.lhs == can.gts == can_lhs
assert can.rhs == can.lts == can_rhs

can = LMI_NSD(c, m).canonical()
assert can.lhs == can.gts == can_lhs
assert can.rhs == can.lts == can_rhs
assert isinstance(can, LMI_PSD)

can = LMI_ND(c, m).canonical()
assert can.lhs == can.gts == can_lhs
assert can.rhs == can.lts == can_rhs
assert isinstance(can, LMI_PD)

### test_LMI_canonical_same()

In [4]:
m = Matrix([[x, y], [y, z+1]])

lmi = LMI_PSD(m)
assert lmi is lmi.canonical()

lmi = LMI_PD(m)
assert lmi is lmi.canonical()

### test_LMI_expanded()

In [5]:
variables = (x, y, z)
m = Matrix([[x, y], [y, z+1]])
c = Matrix([[0, 1], [1, 2]])

expdd = LMI_ND(m+c, c).expanded(variables)
assert expdd == LMI_ND(lm_sym_expanded(m+c, variables), c)

expdd = LMI_PSD(m).expanded(variables)
assert expdd == LMI_PSD(lm_sym_expanded(m, variables))

### test_LMI_PSD_exceptions()

In [6]:
try:
    LMI_PSD(Matrix([[1, x], [y, z]]))
except NonSymmetricMatrixError:
    pass
else:
    assert False

try:
    LMI_PSD(Matrix([[1, x], [y, z]]), assert_symmetry=False)
except NonSymmetricMatrixError:
    assert False

try:
    LMI_PSD(Matrix([[x+y]]), Matrix([[x, y], [y, z+1]]))
except ShapeError:
    pass
else:
    assert False

### test_lmi_latex_printing()

In [7]:
init_lmi_latex_printing()
lmi = LMI_PSD(Matrix([[x, y], [y, z+1]]))
assert r"\succeq 0" == latex(lmi)[-9:]